In [ ]:
import os
import csv
from config import database
from sqlalchemy.orm import sessionmaker
from database.tools import init_engine
from database.models import (Base, Participant, Case, Control, Can, Dors, Sv, Ov)
import pandas as pd
import numpy as np

# Upload to database
Upload data to database from csv files. This only works with a mysql user that has adequate privileges.

In [ ]:
engine = init_engine(database["username"],
                     database["password"],
                     database["dbname"])

if engine.has_table("control"):
    Control.__table__.drop(bind=engine)

if engine.has_table("case"):
    Case.__table__.drop(bind=engine)

Base.metadata.create_all(engine)
Base.prepare(engine)
Session = sessionmaker(engine)
session = Session()

In [ ]:
cases = pd.read_csv(os.path.join(datadir_swedish, "cases.csv"))
controls = pd.read_csv(os.path.join(datadir_swedish, "controls.csv"))

controls.columns = ["participant_lopnr", "case_lopnr"] + [val.lower() for val in controls.columns[2:]]
cases.columns = ["participant_lopnr"] + [val.lower() for val in cases.columns[1:]]

all_lopnr = cases.participant_lopnr.append(controls.participant_lopnr)
all_lopnr = all_lopnr.groupby(all_lopnr).first().astype(int)
all_lopnr.index = np.arange(len(all_lopnr))
participant = pd.DataFrame({"lopnr": all_lopnr})

In [ ]:
participant.to_sql("participant", engine, if_exists="append", chunksize=2**8, index=False)

In [ ]:
cases.to_sql("case", engine, if_exists="append", chunksize=2**8, index=False)
controls.to_sql("control", engine, if_exists="append", chunksize=2**8, index=False)

In [ ]:
ov = pd.read_csv(os.path.join(datadir_swedish, "ov.csv"), nrows=100000)
sv = pd.read_csv(os.path.join(datadir_swedish, "sv.csv"), nrows=100000)
can = pd.read_csv(os.path.join(datadir_swedish, "can.csv"), nrows=100000)
dors = pd.read_csv(os.path.join(datadir_swedish, "dors.csv"), nrows=100000, encoding="latin-1")

In [ ]:
sv.columns = ["participant_lopnr"] + [val.lower() for val in sv.columns[1:]]
(~sv.participant_lopnr.isin(participant.lopnr)).sum()
sv.to_sql("sv", engine, if_exists="append", chunksize=2**8, index=False)

In [ ]:
ov[~sv.participant_lopnr.isin(participant.lopnr)][:10]

In [ ]:
c = controls.groupby("lopnr").first()
c.index.get_level_values("lopnr").isin(cases.lopnr).sum()

In [ ]:
cases.to_sql("case", engine, if_exists="append", chunksize=20000, index=False)
controls.to_sql("control", engine, if_exists="append", chunksize=20000, index=False)

In [ ]:
len(controls.groupby("lopnr")), len(controls)